In [8]:
from transformers import pipeline
# sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis", return_all_scores=True)
sentiment_pipeline = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)


Device set to use cuda:0
/home/biggergig/micromamba/envs/fairai/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [9]:

data = ["She was super depressed", "He was very happy"]
sentiment_pipeline(data)

[[{'label': 'sadness', 'score': 0.21912488341331482},
  {'label': 'joy', 'score': 0.7152641415596008},
  {'label': 'love', 'score': 0.0020415394101291895},
  {'label': 'anger', 'score': 0.05050911381840706},
  {'label': 'fear', 'score': 0.009933545254170895},
  {'label': 'surprise', 'score': 0.003126824740320444}],
 [{'label': 'sadness', 'score': 0.0005159176653251052},
  {'label': 'joy', 'score': 0.9983043670654297},
  {'label': 'love', 'score': 0.0004944835673086345},
  {'label': 'anger', 'score': 0.0003703442052938044},
  {'label': 'fear', 'score': 0.0001525327388662845},
  {'label': 'surprise', 'score': 0.00016229950415436178}]]